# Protocol timer

## Initialization

### Load Libraries

In [6]:
from IPython.core.getipython import get_ipython
from IPython.display import display, Markdown, Latex
import pandas as pd
from time import time, sleep
from tqdm.notebook import tqdm
from tqdm import trange
import datetime as date
import requests
import json

# from tqdm.contrib.telegram import tqdm, trange
from configparser import ConfigParser


from timer_functions import *

In [7]:
parser = ConfigParser()

_ = parser.read('token.secrets')

channel_id = parser.get('discord', 'channel_id')

webhook_url = parser.get('discord', 'webhook')

matrix_webhook_url = parser.get('matrix', 'webhook')

In [8]:
# Define functions used in this notebook
def create_new_cell(contents):
    shell = get_ipython()

    payload = dict(
        source="set_next_input",
        text=contents,
        replace=False,
    )
    shell.payload_manager.write_payload(payload, single=False)


def calculate_seconds(timestring):
    time_obj = date.datetime.strptime(timestring, "%H:%M:%S")
    hours = time_obj.hour
    minutes = time_obj.minute
    seconds = time_obj.second

    # Calculate the total number of seconds
    total_seconds = hours * 3600 + minutes * 60 + seconds

    return total_seconds


def step_commands(df, index_num):
    line1 = f"print('**{df.Combined.iloc[index_num]}**')"
    total_seconds = calculate_seconds(df.Duration.iloc[index_num])

    line2 = f"print('Duration is {round(total_seconds/60, 3)} minutes')"

    line3 = f"countdown(df.Duration.iloc[{index_num}], df.Combined.iloc[{index_num}], df.Duration.iloc[{index_num}])"

    combined_lines = line1 + " \n" + line2 + " \n" + line3

    return combined_lines


def countdown(timestring, step_text, duration):
    display(f"Starting {step_text}")
    # send_discord_message(f"Starting {step_text}.")
    send_matrix_message(step_text, duration)

    time_obj = date.datetime.strptime(timestring, "%H:%M:%S")

    hours = time_obj.hour
    minutes = time_obj.minute
    seconds = time_obj.second

    # Calculate the total number of seconds
    total_seconds = hours * 3600 + minutes * 60 + seconds

    # While loop that checks if total_seconds reaches zero
    # If not zero, decrement total time by one second
    total_seconds_original = total_seconds

    pbar = tqdm(
        # total=100,
        total=total_seconds,
        bar_format="{desc}   {percentage:3.0f}% [Elapsed: {elapsed} -- Remaining: {remaining}]",
    )
    while total_seconds > 0:
        # # Timer represents time left on countdown
        timer = date.timedelta(seconds=total_seconds)

        # print(timer, sep='\r')

        # Update the tqdm progress bar
        # pbar.update(100 / total_seconds_original)

        sleep(1)

        total_seconds -= 1
        pbar.update(1)

    pbar.refresh()
    display(f"COMPLETED {step_text}")
    send_discord_message(f"COMPLETED {step_text}.")


def compose_discord_json(message):
    json_payload = {
        "content": message,
        "username": "Protocol Timer",
        "avatar_url": "https://raw.githubusercontent.com/github/explore/149e057770c384ddaba0393b369c2c8f16e433bb/topics/jupyter-notebook/jupyter-notebook.png",
        "tts": True,
        "embeds": [
            {
                "title": "Protocol Timer",
                "type": "rich",
                "timestamp": date.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ"),
                "fields": [
                    {
                        "name": "Protocol Step",
                        "value": message,
                    },
                ],
                "footer": {
                    "text": "Protocol-Timer created by Pranav Mishra (pranavmishra90/protocol-timer)"
                },
            }
        ],
    }

    return json_payload

def compose_matrix_json(message, duration):
    duration_parts = duration.split(":")
    duration_var = date.timedelta(hours=int(duration_parts[0]), minutes=int(duration_parts[1]), seconds=int(duration_parts[2]))

    current_datetime = date.datetime.now()
    resulting_datetime = current_datetime + duration_var

    step_end_time = resulting_datetime.strftime("%I:%M %p")

    json_payload = {
        "html": f"<h2>{message}</h2><b>Duration</b>: {duration}</br><b>End Time</b>: {step_end_time}"
    }
    return json_payload



def send_matrix_message(message, duration):
    json_payload = compose_matrix_json(message, duration)
    headers = {"Content-Type": "application/json"}

    response = requests.post(
        matrix_webhook_url, data=json.dumps(json_payload), headers=headers
    )

    if response.status_code == 202:
        print("Matrix notification successful")
    else:
        print(f"Failed to notify on Matrix. Status code: {response.status_code}")

def send_discord_message(message):
    json_payload = compose_discord_json(message)

    headers = {"Content-Type": "application/json"}

    response = requests.post(
        webhook_url, data=json.dumps(json_payload), headers=headers
    )

    if response.status_code == 204:
        print("Discord notification successful!")
    else:
        print(f"Failed to notify on discord. Status code: {response.status_code}")


def current_datetime():
    current_datetime = date.datetime.now()

    # Format the datetime as "Day, mm/dd/yy at hh:mm:ss AM/PM"
    formatted_datetime = current_datetime.strftime("%A, %m/%d/%y at %I:%M:%S %p")

    return formatted_datetime

## Import the protocol and format the dataframe

In [9]:
df = pd.read_csv('./protocol.csv')

#flip the dataframe since a new "step" is always inserted immediately after the currently running one (new cells end up in reverse order)
df = df.iloc[::-1].reset_index()
df.drop(columns='index', inplace=True)

df['Combined'] = "Step "+ df['Step Number'].astype(str) + ": " + df['Step Text'].astype(str)

df = df.astype({'Step Number': 'int', 'Step Text': 'string', 'Duration': 'string', 'Combined': 'string'})

display(df)

,Step Number,Step Text,Duration,Comment,Combined
0,15,Paraffin (60 C),1:30:00,NaN,Step 15: Paraffin (60 C)
1,14,Paraffin (60 C),1:00:00,NaN,Step 14: Paraffin (60 C)
2,13,Paraffin (60 C),0:30:00,NaN,Step 13: Paraffin (60 C)
3,12,Xylene (RT),1:00:00,NaN,Step 12: Xylene (RT)
4,11,Xylene (RT),0:40:00,NaN,Step 11: Xylene (RT)
5,10,Xylene (RT),0:30:00,NaN,Step 10: Xylene (RT)
6,9,Xylene (RT),0:30:00,NaN,Step 9: Xylene (RT)
7,8,100% Ethanol (RT),1:00:00,NaN,Step 8: 100% Ethanol (RT)
8,7,100% Ethanol (RT),0:50:00,NaN,Step 7: 100% Ethanol (RT)
9,6,100% Ethanol (RT),0:45:00,NaN,Step 6: 100% Ethanol (RT)


## Your Protocol
**Instructions**
A cell has been created for each step of your protocol according to the durations specified. When you "run" a cell, a progress bar will appear, with a timer for the elapsed time and a time remaining countdown.

After the timer runs out, a completed message will be displayed.

### Run the next cell *once*

In [10]:
create_new_cell("#------ End of protocol ------# \n #When you have reached this point, you can delete all of the protocol steps which have been created, or simply close this file without saving")

create_new_cell('send_discord_message(f"---- PROTOCOL COMPLETE at {current_datetime()} ----")')

for row in df.index:
    all_in_one = step_commands(df, row)
    
    create_new_cell(all_in_one)

display(Markdown('## Start the protocol\n**When you are ready to begin, run the cell below**'))

create_new_cell('send_discord_message(f"---- STARTING PROTOCOL at {current_datetime()} ----")')

create_new_cell("#------ Click here, then run each cells below ------#")

## Start the protocol
**When you are ready to begin, run the cell below**

In [ ]:
#------ Click here, then run each cells below ------#